Metaboanalyst script for running PLS-DA and getting VIP scores for lipids

In [2]:
# Install all packages and then metaboanalyst 4.0

# 1. Install the manager for biology packages
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

# 2. Create the list of tools needed
packages <- c("impute", "pcaMethods", "globaltest", "GlobalAncova", "Rgraphviz", 
              "preprocessCore", "genefilter", "SSPA", "sva", "limma", 
              "KEGGgraph", "siggenes", "BiocParallel", "MSnbase", 
              "multtest", "RBGL", "edgeR", "fgsea")

# 3. Install them all at once
BiocManager::install(packages)

Bioconductor version '3.19' is out-of-date; the current release version '3.22'
  is available with R version '4.5'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.19 (BiocManager 1.30.25), R 4.4.3 (2025-02-28 ucrt)

Warning message:
"package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'impute' 'pcaMethods' 'preprocessCore' 'limma'
  'BiocParallel' 'MSnbase' 'edgeR' 'fgsea'"
Installing package(s) 'globaltest', 'GlobalAncova', 'Rgraphviz', 'genefilter',
  'SSPA', 'sva', 'KEGGgraph', 'siggenes', 'multtest', 'RBGL'

Warning message:
"package 'SSPA' is not available for Bioconductor version '3.19'

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manual

package 'annotate' successfully unpacked and MD5 sums checked
package 'GSEABase' successfully unpacked and MD5 sums checked
package 'VGAM' successfully unpacked and MD5 sums checked
package 'graph' successfully unpacked and MD5 sums checked
package 'scrime' successfully unpacked and MD5 sums checked
package 'globaltest' successfully unpacked and MD5 sums checked
package 'GlobalAncova' successfully unpacked and MD5 sums checked
package 'Rgraphviz' successfully unpacked and MD5 sums checked
package 'genefilter' successfully unpacked and MD5 sums checked
package 'sva' successfully unpacked and MD5 sums checked
package 'KEGGgraph' successfully unpacked and MD5 sums checked
package 'siggenes' successfully unpacked and MD5 sums checked
package 'multtest' successfully unpacked and MD5 sums checked
package 'RBGL' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\husse\AppData\Local\Temp\RtmpiOqI07\downloaded_packages


Installation paths not writeable, unable to update packages
  path: C:/Program Files/R/R-4.4.3/library
  packages:
    boot, cluster, foreign, lattice, MASS, Matrix, mgcv, nlme, survival

Old packages: 'admisc', 'anytime', 'aplot', 'base64enc', 'BH', 'BiocManager',
  'blob', 'bookdown', 'broom', 'bslib', 'car', 'carData', 'circlize', 'cli',
  'collections', 'colorspace', 'commonmark', 'cowplot', 'cpp11', 'credentials',
  'data.table', 'dbplyr', 'dendextend', 'Deriv', 'devtools', 'diffobj',
  'digest', 'doBy', 'doRNG', 'downlit', 'dplyr', 'DT', 'dtplyr', 'e1071',
  'emdbook', 'emmeans', 'export', 'FactoMineR', 'fansi', 'fastmatch',
  'fitdistrplus', 'flextable', 'forcats', 'futile.logger', 'future',
  'future.apply', 'gam', 'gargle', 'gdtools', 'gert', 'GetoptLong', 'ggforce',
  'ggfun', 'ggnewscale', 'ggplot2', 'ggplotify', 'ggpubr', 'ggraph',
  'ggridges', 'ggsci', 'ggVennDiagram', 'gh', 'glmnet', 'GlobalOptions',
  'globals', 'gmm', 'googledrive', 'gplots', 'gprofiler2', 'here', 'hms

In [3]:
# Step 1: Install devtools
install.packages("devtools")
library(devtools)

# Step 2: Install MetaboAnalystR with documentation
devtools::install_github("xia-lab/MetaboAnalystR", build = TRUE, build_vignettes = TRUE, build_manual =T)

Installing package into 'C:/Users/husse/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

also installing the dependencies 'rprojroot', 'usethis', 'cli', 'miniUI', 'pkgbuild', 'pkgdown', 'pkgload', 'rlang', 'roxygen2'


Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/4.4:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/4.4/PACKAGES'"


package 'rprojroot' successfully unpacked and MD5 sums checked
package 'usethis' successfully unpacked and MD5 sums checked
package 'cli' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'cli'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\husse\AppData\Local\R\win-library\4.4\00LOCK\cli\libs\x64\cli.dll to C:\Users\husse\AppData\Local\R\win-library\4.4\cli\libs\x64\cli.dll: Permission denied"
Warning message:
"restored 'cli'"


package 'miniUI' successfully unpacked and MD5 sums checked
package 'pkgbuild' successfully unpacked and MD5 sums checked
package 'pkgdown' successfully unpacked and MD5 sums checked
package 'pkgload' successfully unpacked and MD5 sums checked
package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\husse\AppData\Local\R\win-library\4.4\00LOCK\rlang\libs\x64\rlang.dll to C:\Users\husse\AppData\Local\R\win-library\4.4\rlang\libs\x64\rlang.dll: Permission denied"
Warning message:
"restored 'rlang'"


package 'roxygen2' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'roxygen2'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\husse\AppData\Local\R\win-library\4.4\00LOCK\roxygen2\libs\x64\roxygen2.dll to C:\Users\husse\AppData\Local\R\win-library\4.4\roxygen2\libs\x64\roxygen2.dll: Permission denied"
Warning message:
"restored 'roxygen2'"


package 'devtools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\husse\AppData\Local\Temp\RtmpiOqI07\downloaded_packages


Loading required package: usethis



ERROR: Error: package or namespace load failed for 'devtools' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'cli' 3.6.4 is already loaded, but >= 3.6.5 is required
